## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
tf.set_random_seed(5)

In [0]:
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [0]:
!pip3 install -U tensorflow --quiet

     |████████████████████████████████| 86.3MB 137kB/s 
     |████████████████████████████████| 3.8MB 32.1MB/s 
     |████████████████████████████████| 450kB 49.9MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.


In [0]:
tf.__version__

'1.15.0'

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TensorFlow/prices.csv')

### Check all columns in the dataset

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [0]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
open,851264.0,7.083699e+01,8.369588e+01,0.85,3.384000e+01,5.277000e+01,7.988000e+01,1.584440e+03
close,851264.0,7.085711e+01,8.368969e+01,0.86,3.385000e+01,5.280000e+01,7.989000e+01,1.578130e+03
low,851264.0,7.011841e+01,8.287729e+01,0.83,3.348000e+01,5.223000e+01,7.911000e+01,1.549940e+03
high,851264.0,7.154348e+01,8.446550e+01,0.88,3.419000e+01,5.331000e+01,8.061000e+01,1.600930e+03
volume,851264.0,5.415113e+06,1.249468e+07,0.00,1.221500e+06,2.476250e+06,5.222500e+06,8.596434e+08


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'], axis =1, inplace =True)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
new_data = data.loc[0:999,:]

In [0]:
new_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float64
close     1000 non-null float64
low       1000 non-null float64
high      1000 non-null float64
volume    1000 non-null float64
dtypes: float64(5)
memory usage: 39.2 KB


In [0]:
new_data.head()
new_data['volume']=new_data['volume'].div(1000000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
new_data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
x = new_data.drop('volume', axis =1)

In [0]:
y = new_data['volume']

In [0]:
xtrain,  xtest, ytrain, ytest = train_test_split(x,y,test_size =0.3,random_state =5)

In [0]:
train_x = np.array(xtrain).astype('float32')
train_y = np.array(ytrain).astype('float32')
test_x = np.array(xtest).astype('float32')
test_y = np.array(ytest).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.fit_transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(train_x, train_y, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(train_y, prediction(train_x, w, b)))

Current Loss on iteration 0 Tensor("Mean_3:0", shape=(), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_5:0", shape=(), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_7:0", shape=(), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_9:0", shape=(), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_11:0", shape=(), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_13:0", shape=(), dtype=float32)
Current Loss on iteration 6 Tensor("Mean_15:0", shape=(), dtype=float32)
Current Loss on iteration 7 Tensor("Mean_17:0", shape=(), dtype=float32)
Current Loss on iteration 8 Tensor("Mean_19:0", shape=(), dtype=float32)
Current Loss on iteration 9 Tensor("Mean_21:0", shape=(), dtype=float32)
Current Loss on iteration 10 Tensor("Mean_23:0", shape=(), dtype=float32)
Current Loss on iteration 11 Tensor("Mean_25:0", shape=(), dtype=float32)
Current Loss on iteration 12 Tensor("Mean_27:0", shape=(), dtype=float32)
Current Loss on iteration 13 Tensor("Mean_29:0", sha

### Get the shapes and values of W and b

In [0]:
#Check Weights and Bias
print('Weights:\n', w)
print('Bias:\n',b)

Weights:
 Tensor("sub_401:0", shape=(4, 1), dtype=float32)
Bias:
 Tensor("sub_402:0", shape=(1,), dtype=float32)


In [0]:
train_x[0]

array([0.50204444, 0.50088656, 0.4944026 , 0.5026234 ], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [0]:
with tf.Session() as sess:  
  val_2 =(prediction(test_x, w, b).eval()) 


In [0]:
val_2[0]

array([5.0377603], dtype=float32)

In [0]:
test_y[0]

38.4091

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [177]:
iris = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TensorFlow/Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [178]:
df_occ = pd.get_dummies(data = iris['Species'], dummy_na = True)
df_occ = df_occ.rename(columns = {'Iris-setosa' : 'setosa','Iris-versicolor' : 'versicolor','Iris-virginica' : 'virginica'})
iris = pd.concat([iris, df_occ.iloc[:, 0:3]], axis = 1)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,setosa,versicolor,virginica
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0


In [0]:
iris.drop(['Species'], axis=1, inplace =True)

In [180]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,setosa,versicolor,virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
x = iris.drop(['setosa','versicolor','virginica'], axis=1)
x.drop(['Id'], axis=1, inplace =True)
y = iris[['setosa','versicolor','virginica']]

In [191]:
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [192]:
y.head()

,setosa,versicolor,virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [193]:
x.shape

(150, 4)

In [194]:
y.shape

(150, 3)

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
xtrain,  xtest, ytrain, ytest = train_test_split(x,y,test_size =0.2,random_state =5)

In [197]:
xtrain.shape

(120, 4)

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1,input_shape=(4,)))


#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [204]:
model.fit(xtrain, ytrain, 
          validation_data=(xtest, ytest), 
          epochs=50,
          batch_size=15)

Train on 120 samples, validate on 30 samples
Epoch 1/50
120/120 [==============================] - 0s 352us/sample - loss: 0.4751 - acc: 0.9167 - val_loss: 0.7664 - val_acc: 0.6333
Epoch 2/50
120/120 [==============================] - 0s 352us/sample - loss: 0.4759 - acc: 0.8750 - val_loss: 0.6975 - val_acc: 0.6667
Epoch 3/50
120/120 [==============================] - 0s 315us/sample - loss: 0.4500 - acc: 0.8833 - val_loss: 0.6403 - val_acc: 0.6667
Epoch 4/50
120/120 [==============================] - 0s 319us/sample - loss: 0.4600 - acc: 0.9083 - val_loss: 0.5948 - val_acc: 0.6667
Epoch 5/50
120/120 [==============================] - 0s 337us/sample - loss: 0.4580 - acc: 0.8917 - val_loss: 0.5580 - val_acc: 0.6667
Epoch 6/50
120/120 [==============================] - 0s 353us/sample - loss: 0.4427 - acc: 0.8750 - val_loss: 0.5263 - val_acc: 0.6667
Epoch 7/50
120/120 [==============================] - 0s 347us/sample - loss: 0.4733 - acc: 0.8500 - val_loss: 0.5000 - val_acc: 0.6667
Epo

### Model Prediction

In [208]:
ypred= model.predict(xtest)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 5         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1)                 4         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 6         
Total params: 15
Trainable params: 13
Non-trainable params: 2
_________________________________________________________________


In [214]:
ypred[3].argmax()

0

### Save the Model

In [0]:
model.save('R6_iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?